##### Copyright 2021 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# SNGP를 사용한 불확실성 인식 딥 러닝

<table class="tfo-notebook-buttons" align="left">
  <td><a target="_blank" href="https://www.tensorflow.org/tutorials/understanding/sngp"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">TensorFlow.org에서 보기</a></td>
  <td><a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/ko/tutorials/understanding/sngp.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Google Colab에서 실행하기</a></td>
  <td><a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/ko/tutorials/understanding/sngp.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">GitHub에서 보기</a></td>
  <td><a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/ko/tutorials/understanding/sngp.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">노트북 다운로드하기</a></td>
</table>

의료 의사 결정 및 자율 주행과 같이 안전이 중요한 AI 애플리케이션 또는 데이터에 본질적으로 노이즈가 많은 경우(예: 자연어 이해) 딥 분류기가 불확실성을 안정적으로 정량화하는 것이 중요합니다. 딥 분류기는 자체적인 한계와 언제 인간 전문가에게 제어권을 넘겨야 하는지 알 수 있어야 합니다. 이 튜토리얼은 **스펙트럼 정규화 신경 가우시안 프로세스([SNGP](https://arxiv.org/abs/2006.10108){.external})**라는 기술을 사용하여 불확실성을 정량화하는 딥 분류기의 능력을 향상시키는 방법을 보여줍니다.

SNGP의 핵심 아이디어는 네트워크에 간단한 수정을 적용하여 딥 분류기의 ***거리 인식***을 향상시키는 것입니다. 모델의 *거리 인식*은 모델의 예측 확률이 테스트 예제와 훈련 데이터 사이의 거리를 어떻게 반영하는지 측정한 것입니다. 이것은 표준 확률 모델(예: RBF 커널이 있는 [가우시안 프로세스](https://en.wikipedia.org/wiki/Gaussian_process){.external})에 일반적이지만 딥 신경망을 사용한 모델에는 부족한 바람직한 속성입니다. SNGP는 예측 정확도를 유지하면서 이 가우시안 프로세스 동작을 딥 분류기에 주입하는 간단한 방법을 제공합니다.

이 튜토리얼은 [scikit-learn의 두 개의 달](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.make_moons.html){.external} 데이터세트에 대한 ResNet(deep residual network) 기반 SNGP 모델을 구현하고 이 모델의 불확실성 표면을 [Monte Carlo 드롭아웃](https://arxiv.org/abs/1506.02142){.external} 및 [딥 앙상블](https://arxiv.org/abs/1612.01474){.external}의 두 가지 주요 불확실성 접근 방식의 불확실성 표면과 비교합니다.

이 튜토리얼은 장난감 2D 데이터세트의 SNGP 모델을 보여줍니다. BERT 기반을 사용하여 실제 자연어 이해 작업에 SNGP를 적용하는 예는 [SNGP-BERT 튜토리얼](https://www.tensorflow.org/text/tutorials/uncertainty_quantification_with_sngp_bert)을 확인하세요. 다양한 벤치마크 데이터세트(예: [CIFAR-100](https://www.tensorflow.org/datasets/catalog/cifar100), [ImageNet](https://www.tensorflow.org/datasets/catalog/imagenet2012), [Jigsaw 독성 감지](https://www.tensorflow.org/datasets/catalog/wikipedia_toxicity_subtypes))에 대한 SNGP 모델(및 기타 여러 불확실성 방법)의 고품질 구현은 [불확실성 베이스라인](https://github.com/google/uncertainty-baselines){.external} 벤치마크를 참조하세요.

## SNGP 소개

SNGP는 유사한 수준의 정확도와 대기 시간을 유지하면서 딥 분류기의 불확실성 품질을 개선하는 간단한 접근 방식입니다. ResNet(deep residual network)가 주어지면 SNGP는 모델에 두 가지 간단한 변경을 수행합니다.

- 숨겨진 잔차 레이어에 스펙트럼 정규화를 적용합니다.
- 밀집 출력 레이어를 가우시안 프로세스 레이어로 대체합니다.

> ![SNGP](http://tensorflow.org/tutorials/understanding/images/sngp.png)


다른 불확실성 접근 방식(예: Monte Carlo 드롭아웃 또는 딥 앙상블)과 비교할 때 SNGP에는 다음과 같은 몇 가지 장점이 있습니다.

- 광범위한 첨단 잔차 기반 아키텍처(예: (Wide) ResNet, DenseNet 또는 BERT)에서 작동합니다.
- 앙상블 평균에 의존하지 않는 단일 모델 방법입니다. 따라서 SNGP는 단일 결정성 네트워크와 유사한 수준의 대기 시간을 가지며 [ImageNet](https://github.com/google/uncertainty-baselines/tree/main/baselines/imagenet){.external} 및 [Jigsaw Toxic Comments 분류](https://github.com/google/uncertainty-baselines/tree/main/baselines/toxic_comments){.external}와 같은 대규모 데이터세트로 쉽게 확장할 수 있습니다.
- *거리 인식* 속성 덕분에 강력한 도메인 외부 감지 성능을 제공합니다.

이 방법의 단점은 다음과 같습니다.

- SNGP의 예측 불확실성은 [라플라스 근사](http://www.gaussianprocess.org/gpml/chapters/RW3.pdf){.external}를 사용하여 계산됩니다. 따라서 이론적으로 SNGP의 사후 불확실성은 정확한 가우시안 프로세스의 불확실성과 다릅니다.

- SNGP 훈련에는 새 epoch가 시작될 때 공분산 재설정 단계가 필요합니다. 이로 인해 훈련 파이프라인의 복잡성이 약간 커질 수 있습니다. 이 튜토리얼은 Keras 콜백을 사용하여 이를 구현하는 간단한 방법을 보여줍니다.

## 설치

In [ ]:
!pip install -U -q --use-deprecated=legacy-resolver tf-models-official tensorflow

In [ ]:
# refresh pkg_resources so it takes the changes into account.
import pkg_resources
import importlib
importlib.reload(pkg_resources)

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.colors as colors

import sklearn.datasets

import numpy as np
import tensorflow as tf

import official.nlp.modeling.layers as nlp_layers

시각화 매크로를 정의합니다.

In [ ]:
plt.rcParams['figure.dpi'] = 140

DEFAULT_X_RANGE = (-3.5, 3.5)
DEFAULT_Y_RANGE = (-2.5, 2.5)
DEFAULT_CMAP = colors.ListedColormap(["#377eb8", "#ff7f00"])
DEFAULT_NORM = colors.Normalize(vmin=0, vmax=1,)
DEFAULT_N_GRID = 100

## 두 개의 달 데이터세트

[scikit-learn 두 개의 달 데이터세트](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.make_moons.html){.external}로부터 훈련 및 평가 데이터세트를 만듭니다.

In [ ]:
def make_training_data(sample_size=500):
  """Create two moon training dataset."""
  train_examples, train_labels = sklearn.datasets.make_moons(
      n_samples=2 * sample_size, noise=0.1)

  # Adjust data position slightly.
  train_examples[train_labels == 0] += [-0.1, 0.2]
  train_examples[train_labels == 1] += [0.1, -0.2]

  return train_examples, train_labels

전체 2D 입력 공간에서 모델의 예측 동작을 평가합니다.

In [ ]:
def make_testing_data(x_range=DEFAULT_X_RANGE, y_range=DEFAULT_Y_RANGE, n_grid=DEFAULT_N_GRID):
  """Create a mesh grid in 2D space."""
  # testing data (mesh grid over data space)
  x = np.linspace(x_range[0], x_range[1], n_grid)
  y = np.linspace(y_range[0], y_range[1], n_grid)
  xv, yv = np.meshgrid(x, y)
  return np.stack([xv.flatten(), yv.flatten()], axis=-1)

모델 불확실성을 평가하려면 세 번째 클래스에 속하는 도메인 외(OOD) 데이터세트를 추가합니다. 모델은 훈련 중에 이러한 OOD 예제를 관찰하지 않습니다.

In [ ]:
def make_ood_data(sample_size=500, means=(2.5, -1.75), vars=(0.01, 0.01)):
  return np.random.multivariate_normal(
      means, cov=np.diag(vars), size=sample_size)

In [ ]:
# Load the train, test and OOD datasets.
train_examples, train_labels = make_training_data(
    sample_size=500)
test_examples = make_testing_data()
ood_examples = make_ood_data(sample_size=500)

# Visualize
pos_examples = train_examples[train_labels == 0]
neg_examples = train_examples[train_labels == 1]

plt.figure(figsize=(7, 5.5))

plt.scatter(pos_examples[:, 0], pos_examples[:, 1], c="#377eb8", alpha=0.5)
plt.scatter(neg_examples[:, 0], neg_examples[:, 1], c="#ff7f00", alpha=0.5)
plt.scatter(ood_examples[:, 0], ood_examples[:, 1], c="red", alpha=0.1)

plt.legend(["Positive", "Negative", "Out-of-Domain"])

plt.ylim(DEFAULT_Y_RANGE)
plt.xlim(DEFAULT_X_RANGE)

plt.show()

여기서 파란색과 주황색은 양수 및 음수 클래스를 나타내고 빨간색은 OOD 데이터를 나타냅니다. 불확실성을 잘 정량화하는 모델은 훈련 데이터에 근접할 때(즉, $p(x_{test})$가 0 또는 1에 가까움) 확실해지고 훈련 데이터 영역에서 멀어질 때(즉, $p(x_{test})$가 0.5에 가까움) 불확실해질 것으로 예상됩니다.

## 결정성 모델

### 모델 정의하기

(베이스라인) 결정성 모델에서 시작: 드롭아웃 정규화가 있는 멀티 레이어 ResNet(residual network).

In [ ]:
#@title
class DeepResNet(tf.keras.Model):
  """Defines a multi-layer residual network."""
  def __init__(self, num_classes, num_layers=3, num_hidden=128,
               dropout_rate=0.1, **classifier_kwargs):
    super().__init__()
    # Defines class meta data.
    self.num_hidden = num_hidden
    self.num_layers = num_layers
    self.dropout_rate = dropout_rate
    self.classifier_kwargs = classifier_kwargs

    # Defines the hidden layers.
    self.input_layer = tf.keras.layers.Dense(self.num_hidden, trainable=False)
    self.dense_layers = [self.make_dense_layer() for _ in range(num_layers)]

    # Defines the output layer.
    self.classifier = self.make_output_layer(num_classes)

  def call(self, inputs):
    # Projects the 2d input data to high dimension.
    hidden = self.input_layer(inputs)

    # Computes the ResNet hidden representations.
    for i in range(self.num_layers):
      resid = self.dense_layers[i](hidden)
      resid = tf.keras.layers.Dropout(self.dropout_rate)(resid)
      hidden += resid

    return self.classifier(hidden)

  def make_dense_layer(self):
    """Uses the Dense layer as the hidden layer."""
    return tf.keras.layers.Dense(self.num_hidden, activation="relu")

  def make_output_layer(self, num_classes):
    """Uses the Dense layer as the output layer."""
    return tf.keras.layers.Dense(
        num_classes, **self.classifier_kwargs)

이 튜토리얼은 128개의 숨겨진 유닛이 있는 6단 레이어 ResNet을 사용합니다.

In [ ]:
resnet_config = dict(num_classes=2, num_layers=6, num_hidden=128)

In [ ]:
resnet_model = DeepResNet(**resnet_config)

In [ ]:
resnet_model.build((None, 2))
resnet_model.summary()

### 모델 훈련하기

`SparseCategoricalCrossentropy`를 손실 함수로 사용하고 Adam 옵티마이저를 사용하도록 훈련 매개변수를 구성합니다.

In [ ]:
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
metrics = tf.keras.metrics.SparseCategoricalAccuracy(),
optimizer = tf.keras.optimizers.legacy.Adam(learning_rate=1e-4)

train_config = dict(loss=loss, metrics=metrics, optimizer=optimizer)

배치 크기가 128인 100회 epoch에 대해 모델을 훈련합니다.

In [ ]:
fit_config = dict(batch_size=128, epochs=100)

In [ ]:
resnet_model.compile(**train_config)
resnet_model.fit(train_examples, train_labels, **fit_config)

### 불확실성 시각화

In [ ]:
#@title
def plot_uncertainty_surface(test_uncertainty, ax, cmap=None):
  """Visualizes the 2D uncertainty surface.
  
  For simplicity, assume these objects already exist in the memory:

    test_examples: Array of test examples, shape (num_test, 2).
    train_labels: Array of train labels, shape (num_train, ).
    train_examples: Array of train examples, shape (num_train, 2).
  
  Arguments:
    test_uncertainty: Array of uncertainty scores, shape (num_test,).
    ax: A matplotlib Axes object that specifies a matplotlib figure.
    cmap: A matplotlib colormap object specifying the palette of the
      predictive surface.

  Returns:
    pcm: A matplotlib PathCollection object that contains the palette
      information of the uncertainty plot.
  """
  # Normalize uncertainty for better visualization.
  test_uncertainty = test_uncertainty / np.max(test_uncertainty)

  # Set view limits.
  ax.set_ylim(DEFAULT_Y_RANGE)
  ax.set_xlim(DEFAULT_X_RANGE)

  # Plot normalized uncertainty surface.
  pcm = ax.imshow(
      np.reshape(test_uncertainty, [DEFAULT_N_GRID, DEFAULT_N_GRID]),
      cmap=cmap,
      origin="lower",
      extent=DEFAULT_X_RANGE + DEFAULT_Y_RANGE,
      vmin=DEFAULT_NORM.vmin,
      vmax=DEFAULT_NORM.vmax,
      interpolation='bicubic',
      aspect='auto')

  # Plot training data.
  ax.scatter(train_examples[:, 0], train_examples[:, 1],
             c=train_labels, cmap=DEFAULT_CMAP, alpha=0.5)
  ax.scatter(ood_examples[:, 0], ood_examples[:, 1], c="red", alpha=0.1)

  return pcm

이제 결정성 모델의 예측을 시각화합니다. 먼저 클래스 확률을 플로팅합니다: $$p(x) = softmax(logit(x))$$

In [ ]:
resnet_logits = resnet_model(test_examples)
resnet_probs = tf.nn.softmax(resnet_logits, axis=-1)[:, 0]  # Take the probability for class 0.

In [ ]:
_, ax = plt.subplots(figsize=(7, 5.5))

pcm = plot_uncertainty_surface(resnet_probs, ax=ax)

plt.colorbar(pcm, ax=ax)
plt.title("Class Probability, Deterministic Model")

plt.show()

이 플롯에서 노란색과 자주색은 두 클래스에 대한 예측 확률입니다. 이 결정성 모델은 두 개의 알려진 클래스(파란색과 주황색)를 비선형 결정 경계로 분류하는 작업을 효과적으로 수행했습니다. 그러나 이는 **거리를 인식**하지 않으며 관찰되지 않은 빨간색 도메인 외(OOD) 예제를 주황색 클래스로 확실하게 분류했습니다.

[예측 분산](https://en.wikipedia.org/wiki/Bernoulli_distribution#Variance)을 계산하여 모델 불확실성 시각화: $$var(x) = p(x) * (1 - p(x))$$

In [ ]:
resnet_uncertainty = resnet_probs * (1 - resnet_probs)

In [ ]:
_, ax = plt.subplots(figsize=(7, 5.5))

pcm = plot_uncertainty_surface(resnet_uncertainty, ax=ax)

plt.colorbar(pcm, ax=ax)
plt.title("Predictive Uncertainty, Deterministic Model")

plt.show()

이 플롯에서 노란색은 높은 불확실성을 나타내고 보라색은 낮은 불확실성을 나타냅니다. 결정성 ResNet의 불확실성은 결정 경계로부터 테스트 예제의 거리에만 의존합니다. 이로 인해 훈련 영역을 벗어날 때 모델이 과도한 확신을 갖게 됩니다. 다음 섹션에서는 이 데이터세트에서 SNGP가 어떻게 다르게 작동하는지 보여줍니다.

## SNGP 모델

### SNGP 모델 정의

이제 SNGP 모델을 구현해 보겠습니다. SNGP 구성 요소인 `SpectralNormalization` 및 `RandomFeatureGaussianProcess`는 모두 tensorflow_model의 [내장 레이어](https://github.com/tensorflow/models/tree/master/official/nlp/modeling/layers)에서 사용할 수 있습니다. 

> ![SNGP](http://tensorflow.org/tutorials/understanding/images/sngp.png)


이 두 구성 요소를 자세히 살펴보겠습니다. [전체 SNGP 모델](#full-sngp-model) 섹션으로 이동하여 SNGP의 구현 방법을 알아볼 수도 있습니다.

#### `SpectralNormalization` 래퍼

[`SpectralNormalization`](https://github.com/tensorflow/models/blob/master/official/nlp/modeling/layers/spectral_normalization.py){.external}은 Keras 레이어 래퍼입니다. 다음과 같이 기존 밀집 레이어에 적용할 수 있습니다.

In [ ]:
dense = tf.keras.layers.Dense(units=10)
dense = nlp_layers.SpectralNormalization(dense, norm_multiplier=0.9)

스펙트럼 정규화는 스펙트럼 표준(즉, $W$의 가장 큰 고유값)을 목표 값 `norm_multiplier`로 점진적으로 유도하여 숨겨진 가중치 $W$를 정규화합니다.


참고: 일반적으로 `norm_multiplier`를 1보다 작은 값으로 설정하는 것이 좋습니다. 그러나 실제로는 딥 네트워크가 충분한 표현력을 갖도록 더 큰 값으로 완화할 수도 있습니다.

#### 가우시안 프로세스(GP) 레이어

[`RandomFeatureGaussianProcess`](https://github.com/tensorflow/models/blob/master/official/nlp/modeling/layers/gaussian_process.py){.external}는 딥 신경망으로 엔드 투 엔드 훈련이 가능한 가우시안 프로세스 모델에 [무작위 특성 기반 근사](https://people.eecs.berkeley.edu/~brecht/papers/07.rah.rec.nips.pdf){.external}를 구현합니다. 가우시안 프로세스 레이어는 내부에서 2단 레이어 네트워크를 구현합니다.

```
$$logits(x) = \Phi(x) \beta, \quad \Phi(x)=\sqrt{\frac{2}{M}} * cos(Wx + b)$$
```

Here, $x$ is the input, and $W$ and $b$ are frozen weights initialized randomly from Gaussian and Uniform distributions, respectively. (Therefore, $\Phi(x)$ are called "random features".) $\beta$ is the learnable kernel weight similar to that of a Dense layer. 

In [ ]:
batch_size = 32
input_dim = 1024
num_classes = 10

In [ ]:
gp_layer = nlp_layers.RandomFeatureGaussianProcess(units=num_classes,
                                               num_inducing=1024,
                                               normalize_input=False,
                                               scale_random_features=True,
                                               gp_cov_momentum=-1)

GP 레이어의 주요 매개변수는 다음과 같습니다.

- `units`: 출력 로짓의 차원입니다.
- `num_inducing`: 숨겨진 가중치 $W$의 차원 $M$입니다. 기본값은 1024입니다.
- `normalize_input`: 입력 $x$에 레이어 정규화를 적용할지 여부를 나타냅니다.
- `scale_random_features`: 숨겨진 출력에 $\sqrt{2/M}$ 척도를 적용할지 여부를 나타냅니다.


참고: 학습률에 민감한 딥 신경망(예: ResNet-50 및 ResNet-110)의 경우 일반적으로 `normalize_input=True`로 설정하여 훈련을 안정화하고 `scale_random_features=False`로 설정하여 GP 레이어를 통과할 때 학습률이 예상치 못하게 수정되는 것을 방지하는 것이 좋습니다.

- `gp_cov_momentum`은 모델 공분산이 계산되는 방식을 제어합니다. 양수 값(예: `0.999`)으로 설정하면 모멘텀 기반 이동 평균 업데이트(배치 정규화와 유사)를 사용하여 공분산 행렬이 계산됩니다. `-1`로 설정하면 공분산 행렬이 모멘텀 없이 업데이트됩니다.

참고: 모멘텀 기반 업데이트 방법은 배치 크기에 민감할 수 있습니다. 따라서 일반적으로 공분산을 정확하게 계산하려면 `gp_cov_momentum=-1`을 설정하는 것이 좋습니다. 이것이 제대로 작동하려면 동일한 데이터를 두 번 계산하지 않도록 새 epoch가 시작될 때 공분산 행렬 추정기를 재설정해야 합니다. `RandomFeatureGaussianProcess`의 경우 이를 위해 `reset_covariance_matrix()`를 호출할 수 있습니다. 다음 섹션에서는 Keras의 내장 API를 사용하여 이를 쉽게 구현하는 방법을 보여줍니다.


형상이 `(batch_size, input_dim)`인 배치 입력이 주어지면 GP 레이어는 예측을 위한 `logits` 텐서(형상 `(batch_size, num_classes)`), 그리고 배치 로짓의 사후 공분산 행렬인 `covmat` 텐서(형상 `(batch_size, batch_size)`)를 반환합니다.

In [ ]:
embedding = tf.random.normal(shape=(batch_size, input_dim))

logits, covmat = gp_layer(embedding)

참고: 이 SNGP 모델 구현에서 모든 클래스에 대한 예측 로짓 $logit(x_{test})$는 훈련 데이터에서 $x_{test}$ 사이의 거리를 설명하는 동일한 공분산 행렬인 $var(x_{test})$를 공유합니다.

이론적으로, 다양한 클래스에 대해 서로 다른 분산 값을 계산하도록 알고리즘을 확장할 수 있습니다([원본 SNGP 논문](https://arxiv.org/abs/2006.10108){.external}에 소개된 대로). 그러나 이것을 출력 공간이 큰 문제(예: ImageNet을 사용한 분류 또는 언어 모델링)로 확장하기는 어렵습니다.

<a name="full-sngp-model"></a>

#### 전체 SNGP 모델

기본 클래스 `DeepResNet`이 주어지면 SNGP 모델은 잔여 네트워크의 숨겨진 레이어와 출력 레이어를 수정하여 쉽게 구현할 수 있습니다. `model.fit()` API와의 호환성을 위해 훈련 중에 `logits`만 출력하도록 모델의 `call()` 메서드도 수정합니다.

In [ ]:
class DeepResNetSNGP(DeepResNet):
  def __init__(self, spec_norm_bound=0.9, **kwargs):
    self.spec_norm_bound = spec_norm_bound
    super().__init__(**kwargs)

  def make_dense_layer(self):
    """Applies spectral normalization to the hidden layer."""
    dense_layer = super().make_dense_layer()
    return nlp_layers.SpectralNormalization(
        dense_layer, norm_multiplier=self.spec_norm_bound)

  def make_output_layer(self, num_classes):
    """Uses Gaussian process as the output layer."""
    return nlp_layers.RandomFeatureGaussianProcess(
        num_classes,
        gp_cov_momentum=-1,
        **self.classifier_kwargs)

  def call(self, inputs, training=False, return_covmat=False):
    # Gets logits and a covariance matrix from the GP layer.
    logits, covmat = super().call(inputs)

    # Returns only logits during training.
    if not training and return_covmat:
      return logits, covmat

    return logits

결정성 모델과 동일한 아키텍처를 사용합니다.

In [ ]:
resnet_config

In [ ]:
sngp_model = DeepResNetSNGP(**resnet_config)

In [ ]:
sngp_model.build((None, 2))
sngp_model.summary()

<a name="covariance-reset-callback"></a> 새 epoch가 시작될 때 공분산 행렬을 재설정하는 Keras 콜백을 구현합니다.

In [ ]:
class ResetCovarianceCallback(tf.keras.callbacks.Callback):

  def on_epoch_begin(self, epoch, logs=None):
    """Resets covariance matrix at the beginning of the epoch."""
    if epoch > 0:
      self.model.classifier.reset_covariance_matrix()

`DeepResNetSNGP` 모델 클래스에 이 콜백을 추가합니다.

In [ ]:
class DeepResNetSNGPWithCovReset(DeepResNetSNGP):
  def fit(self, *args, **kwargs):
    """Adds ResetCovarianceCallback to model callbacks."""
    kwargs["callbacks"] = list(kwargs.get("callbacks", []))
    kwargs["callbacks"].append(ResetCovarianceCallback())

    return super().fit(*args, **kwargs)

### 모델 훈련하기

`tf.keras.model.fit`을 사용하여 모델을 훈련합니다.

In [ ]:
sngp_model = DeepResNetSNGPWithCovReset(**resnet_config)
sngp_model.compile(**train_config)
sngp_model.fit(train_examples, train_labels, **fit_config)

### 불확실성 시각화

먼저 예측 로짓과 분산을 계산합니다.

In [ ]:
sngp_logits, sngp_covmat = sngp_model(test_examples, return_covmat=True)

In [ ]:
sngp_variance = tf.linalg.diag_part(sngp_covmat)[:, None]

<a name="mean-field-logits"></a> 이제 사후 예측 확률을 계산합니다. 확률 모델의 예측 확률을 계산하는 고전적인 방법은 다음과 같이 Monte Carlo 샘플링을 사용하는 것입니다.

```
$$E(p(x)) = \frac{1}{M} \sum_{m=1}^M logit_m(x), $$
```

여기서 $M$은 샘플 크기이고 $logit_m(x)$는 SNGP 사후 $MultivariateNormal$(`sngp_logits`, `sngp_covmat`)의 무작위 샘플입니다. 그러나 이 접근 방식은 자율 주행 또는 실시간 입찰과 같이 대기 시간에 민감한 애플리케이션의 경우 속도가 느릴 수 있습니다. 대신 [평균 필드 방법](https://arxiv.org/abs/2006.07584){.external}을 사용하여 $E(p(x))$를 근사시킬 수 있습니다.

```
$$E(p(x)) \approx softmax(\frac{logit(x)}{\sqrt{1+ \lambda * \sigma^2(x)}})$$
```

where $\sigma^2(x)$ is the SNGP variance, and $\lambda$ is often chosen as $\pi/8$ or $3/\pi^2$.

In [ ]:
sngp_logits_adjusted = sngp_logits / tf.sqrt(1. + (np.pi / 8.) * sngp_variance)
sngp_probs = tf.nn.softmax(sngp_logits_adjusted, axis=-1)[:, 0]

참고: $\lambda$를 고정 값으로 고정하는 대신 하이퍼파라미터로 취급하고 이를 조정하여 모델의 보정 성능을 최적화할 수도 있습니다. 이는 딥 러닝 불확실성 자료에서 [온도 스케일링](http://proceedings.mlr.press/v70/guo17a.html){.external}이라고 알려져 있습니다. 

이 평균 필드 메서드는 내장 함수 `layers.gaussian_process.mean_field_logits`로 구현됩니다.

In [ ]:
def compute_posterior_mean_probability(logits, covmat, lambda_param=np.pi / 8.):
  # Computes uncertainty-adjusted logits using the built-in method.
  logits_adjusted = nlp_layers.gaussian_process.mean_field_logits(
      logits, covmat, mean_field_factor=lambda_param)
  
  return tf.nn.softmax(logits_adjusted, axis=-1)[:, 0]

In [ ]:
sngp_logits, sngp_covmat = sngp_model(test_examples, return_covmat=True)
sngp_probs = compute_posterior_mean_probability(sngp_logits, sngp_covmat)

### SNGP 요약

In [ ]:
#@title

def plot_predictions(pred_probs, model_name=""):
  """Plot normalized class probabilities and predictive uncertainties."""
  # Compute predictive uncertainty.
  uncertainty = pred_probs * (1. - pred_probs)

  # Initialize the plot axes.
  fig, axs = plt.subplots(1, 2, figsize=(14, 5))

  # Plots the class probability.
  pcm_0 = plot_uncertainty_surface(pred_probs, ax=axs[0])
  # Plots the predictive uncertainty.
  pcm_1 = plot_uncertainty_surface(uncertainty, ax=axs[1])

  # Adds color bars and titles.
  fig.colorbar(pcm_0, ax=axs[0])
  fig.colorbar(pcm_1, ax=axs[1])

  axs[0].set_title(f"Class Probability, {model_name}")
  axs[1].set_title(f"(Normalized) Predictive Uncertainty, {model_name}")

  plt.show()  

이제 모든 부분을 합칠 수 있습니다. 훈련, 평가 및 불확실성 계산의 전체 절차를 단 5줄로 수행할 수 있습니다.

In [ ]:
def train_and_test_sngp(train_examples, test_examples):
  sngp_model = DeepResNetSNGPWithCovReset(**resnet_config)

  sngp_model.compile(**train_config)
  sngp_model.fit(train_examples, train_labels, verbose=0, **fit_config)

  sngp_logits, sngp_covmat = sngp_model(test_examples, return_covmat=True)
  sngp_probs = compute_posterior_mean_probability(sngp_logits, sngp_covmat)

  return sngp_probs

In [ ]:
sngp_probs = train_and_test_sngp(train_examples, test_examples)

SNGP 모델의 클래스 확률(왼쪽)과 예측 불확실성(오른쪽)을 시각화합니다.

In [ ]:
plot_predictions(sngp_probs, model_name="SNGP")

클래스 확률 플롯(왼쪽)에서 노란색과 자주색이 클래스 확률임을 상기하십시오. 훈련 데이터 영역에 가까울 때 SNGP는 높은 신뢰도로 예제를 올바르게 분류합니다(즉, 거의 0 또는 1의 확률 할당). 훈련 데이터에서 멀어지면 SNGP는 점차 신뢰가 떨어지고 예측 확률은 0.5에 가까워지는 반면 (정규화된) 모델 불확실성은 1로 상승합니다.

이것을 결정성 모델의 불확실성 표면과 비교합니다. 

In [ ]:
plot_predictions(resnet_probs, model_name="Deterministic")

앞에서 언급했듯이 결정성 모델은 *거리를 인식*하지 않습니다. 불확실성은 결정 경계에서 테스트 예제의 거리로 정의됩니다. 이로 인해 모델은 도메인 외 예제(빨간색)에 대해 과도하게 확신하는 예측을 생성합니다.

## 다른 불확실성 접근 방식과의 비교

이 섹션에서는 SNGP의 불확실성을 [Monte Carlo 드롭아웃](https://arxiv.org/abs/1506.02142){.external} 및 [딥 앙상블](https://arxiv.org/abs/1612.01474){.external}과 비교합니다.

이 두 가지 방법은 모두 결정성 모델의 다중 순방향 통과에 대한 Monte Carlo 평균화를 기반으로 합니다. 먼저 앙상블 크기 $M$을 설정합니다.

In [ ]:
num_ensemble = 10

### Monte Carlo 드롭아웃

드롭아웃 레이어가 있는 훈련된 신경망이 주어지면 Monte Carlo 드롭아웃은 평균 예측 확률을 계산합니다.

```
$$E(p(x)) = \frac{1}{M}\sum_{m=1}^M softmax(logit_m(x))$$
```

by averaging over multiple Dropout-enabled forward passes ${logit_m(x)}_{m=1}^M$.

In [ ]:
def mc_dropout_sampling(test_examples):
  # Enable dropout during inference.
  return resnet_model(test_examples, training=True)

In [ ]:
# Monte Carlo dropout inference.
dropout_logit_samples = [mc_dropout_sampling(test_examples) for _ in range(num_ensemble)]
dropout_prob_samples = [tf.nn.softmax(dropout_logits, axis=-1)[:, 0] for dropout_logits in dropout_logit_samples]
dropout_probs = tf.reduce_mean(dropout_prob_samples, axis=0)

In [ ]:
dropout_probs = tf.reduce_mean(dropout_prob_samples, axis=0)

In [ ]:
plot_predictions(dropout_probs, model_name="MC Dropout")

### 딮 앙상블

딥 앙상블은 딥 러닝 불확실성을 위한 첨단(그러나 비용이 많이 드는) 방식입니다. 딥 앙상블을 훈련하려면 먼저 $M$ 앙상블 구성원을 훈련합니다.

In [ ]:
# Deep ensemble training
resnet_ensemble = []
for _ in range(num_ensemble):
  resnet_model = DeepResNet(**resnet_config)
  resnet_model.compile(optimizer=optimizer, loss=loss, metrics=metrics)
  resnet_model.fit(train_examples, train_labels, verbose=0, **fit_config)

  resnet_ensemble.append(resnet_model)

로짓을 수집하고 평균 예측 확률 $E(p(x)) = \frac{1}{M}\sum_{m=1}^M softmax(logit_m(x))$를 계산합니다.

In [ ]:
# Deep ensemble inference
ensemble_logit_samples = [model(test_examples) for model in resnet_ensemble]
ensemble_prob_samples = [tf.nn.softmax(logits, axis=-1)[:, 0] for logits in ensemble_logit_samples]
ensemble_probs = tf.reduce_mean(ensemble_prob_samples, axis=0)

In [ ]:
plot_predictions(ensemble_probs, model_name="Deep ensemble")

Monte Carlo 드롭아웃 및 딥 앙상블 방법은 모두 결정 경계를 덜 확실하게 만들어 모델의 불확실성 능력을 향상시킵니다. 그러나 둘 모두 거리 인식이 부족한 결정성 딥 네트워크의 한계를 물려받습니다.

## 요약

이 튜토리얼에서는 다음을 수행했습니다.

- 거리 인식을 개선하기 위해 딥 분류기에 SNGP 모델을 구현했습니다.
- Keras `Model.fit` API를 사용하여 SNGP 모델을 전체적으로 훈련했습니다.
- SNGP의 불확실성 동작을 시각화했습니다.
- SNGP, Monte Carlo 드롭아웃 및 딥 앙상블 모델 간의 불확실성 동작을 비교했습니다.

## 리소스 및 추가 자료

- 불확실성 인식 자연어 이해를 위해 BERT 모델에 SNGP를 적용하는 예는 [SNGP-BERT 튜토리얼](https://www.tensorflow.org/text/tutorials/uncertainty_quantification_with_sngp_bert)을 확인하세요.
- 다양한 벤치마크 데이터세트(예: [CIFAR](https://github.com/google/uncertainty-baselines), [ImageNet](https://www.tensorflow.org/datasets/catalog/cifar100), [Jigsaw 독성 감지](https://www.tensorflow.org/datasets/catalog/imagenet2012) 등)에서 SNGP 모델(및 기타 여러 불확실성 방법)을 구현하려면 [Uncertainty Baselines GitHub 리포지토리](https://www.tensorflow.org/datasets/catalog/wikipedia_toxicity_subtypes){.external}로 이동하세요.
- SNGP 방법을 더 깊이 있게 이해하려면 [Simple and Principled Uncertainty Estimation with Deterministic Deep Learning via Distance Awareness](https://arxiv.org/abs/2006.10108){.external} 제목의 논문을 확인하세요.
